In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import os

from opioid_agent import OpioidAgent, OpioidAgentParams
from opioid_model import opioid_XGBoost_model, opioid_mlp_model
from opioid_env import OpioidPrescribeEnv

import opioid_env
import time
from opioid_fairness_metrics import *
from opioid_simulation import *

import sys
sys.path.insert(0, os.getcwd() + '/../')
import vanilla_monte_carlo

In [4]:
simulation_data_dir = "./mimic_data_after_preprocess/simulation_set25.csv"
simulation_data = pd.read_csv(simulation_data_dir, index_col='subject_id')
init_params = {'simulation_data': simulation_data}


xgboost_model_dir = "./mimic-model/models/model_xgboost.pkl"
xgboost_threshold_dir = "./mimic-model/models/thresholds.pkl"
xgboost_precision_dir = "./mimic-model/models/precision_xgboost.pkl"
mlp_model_dir = "./mimic-model/models/model_mlp.pth"
mlp_threshold_dir = "./mimic-model/models/thresholds.pkl"
mlp_precision_dir = "./mimic-model/models/precision_xgboost.pkl"

potential_model_name = ['xgboost', 'mlp']
potential_thresholds = [0.3, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7]
transition_func_type = None
potential_sample_ratio = [0.01]
potential_fairness_requirement = [0.038]
potential_n_time_steps = [5]
potential_n_hosp_mode = ['expectation', 'same', 'random_1', 'random_2']
potential_n_prescription_mode = ['expectation', 'random_1', 'random_2']

# get all possible combinations of parameters
all_params = [potential_model_name, potential_thresholds, potential_sample_ratio,
              potential_fairness_requirement, potential_n_time_steps, 
              potential_n_hosp_mode, potential_n_prescription_mode]
all_params = np.array(np.meshgrid(*all_params)).T.reshape(-1, len(all_params))

all_configs = []
for i in range(len(all_params)):
    model_name = all_params[i][0]
    threshold = float(all_params[i][1])
    sample_ratio = float(all_params[i][2])
    fairness_requirement = float(all_params[i][3])
    n_time_steps = int(all_params[i][4])
    n_hosp_mode = all_params[i][5]
    n_prescription_mode = all_params[i][6]
    if model_name == 'xgboost':
        model_class = opioid_XGBoost_model
        model_dir = xgboost_model_dir
        model_threshold_dir = xgboost_threshold_dir
        model_precision_dir = xgboost_precision_dir
    elif model_name == 'mlp':
        model_class = opioid_mlp_model
        model_dir = mlp_model_dir
        model_threshold_dir = mlp_threshold_dir
        model_precision_dir = mlp_precision_dir
    else:
        raise ValueError("model_name should be xgboost or mlp")
    
    # avg_risk_requirement = AverageRiskScore(fairness_requirement)
    fairness_metric = OpioidUnfairnessScore(fairness_requirement)
    config = OpioidRiskConfiguration(model_class, model_dir, model_threshold_dir,
                                     model_precision_dir, threshold,
                                     transition_func_type, sample_ratio,
                                     fairness_metric, n_time_steps,
                                     n_hosp_mode, n_prescription_mode)
    all_configs.append([i, config])


In [5]:
if __name__ == '__main__':
    stop_criteria = (MaxUnfairnessStopCriteria, 0.05)
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTesting(all_configs[:4], init_params, OpioidMonteCarloSimulation, stop_criteria)
    results0 = vanilla_test.run_testing(n_threads=4, t_limit=60)

Total time used: 12.787373781204224 seconds


In [3]:
len(all_configs)

168

In [4]:
# create a vanilla testing object
import pickle
if __name__ == '__main__':
    stop_criteria = (MaxUnfairnessStopCriteria, 0.02)
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTesting(all_configs[:40], init_params, OpioidMonteCarloSimulation, stop_criteria)
    results0 = vanilla_test.run_testing(n_threads=4, t_limit=1800)
    with open('simulation_results/full0-40_w_utility.pkl', 'wb') as f:
        pickle.dump(results0, f)

    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTesting(all_configs[40:80], init_params, OpioidMonteCarloSimulation, stop_criteria)
    results1 = vanilla_test.run_testing(n_threads=4, t_limit=1800)
    with open('simulation_results/full40-80_w_utility.pkl', 'wb') as f:
        pickle.dump(results1, f)

    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTesting(all_configs[80:120], init_params, OpioidMonteCarloSimulation, stop_criteria)
    results2 = vanilla_test.run_testing(n_threads=4, t_limit=1800)
    with open('simulation_results/full80-120_w_utility.pkl', 'wb') as f:
        pickle.dump(results2, f)

    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTesting(all_configs[120:], init_params, OpioidMonteCarloSimulation, stop_criteria)
    results3 = vanilla_test.run_testing(n_threads=4, t_limit=1800)
    with open('simulation_results/full120-168_w_utility.pkl', 'wb') as f:
        pickle.dump(results3, f)

Total time used: 4418.460410118103 seconds
Total time used: 4788.456923961639 seconds
Total time used: 4708.8239958286285 seconds
Total time used: 5208.484763145447 seconds


In [5]:
results = results0 + results1 + results2 + results3

In [6]:
# save the results using pickle
import pickle
with open('simulation_results/montecarlo_long_w_utility.pkl', 'wb') as f:
    pickle.dump(results, f)